In [1]:
import iisignature
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
from Backtesting.Backtesting import Backtesting
from Brokers.Alpaca.Alpaca_keyes import API_KEY, SECRET_KEY
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit

In [3]:
from datetime import datetime

In [4]:
ticker = "JPM"
data_loader = Backtesting()
start_date = datetime(2023, 1, 1) 
end_date = datetime(2025, 1, 1)

data = data_loader.create_dataframe(
    api_key=API_KEY, 
    secret_key=SECRET_KEY,
    start_date=start_date,
    end_date=end_date,
    interval=TimeFrame(amount = 5, unit = TimeFrameUnit.Minute),
    asset_list=ticker
    )

In [5]:
data

,timestamp,open,high,low,close,volume
0,2023-01-03 09:00:00+00:00,134.90,135.00,134.90,135.00,544.0
1,2023-01-03 11:30:00+00:00,135.30,135.40,135.30,135.40,3010.0
2,2023-01-03 11:40:00+00:00,135.20,135.20,135.20,135.20,1000.0
3,2023-01-03 12:00:00+00:00,134.99,135.21,134.64,134.64,2476.0
4,2023-01-03 12:10:00+00:00,134.66,134.66,134.66,134.66,150.0
...,...,...,...,...,...,...
55731,2024-12-31 22:55:00+00:00,239.90,239.90,239.90,239.90,165.0
55732,2024-12-31 23:15:00+00:00,239.97,239.97,239.97,239.97,301.0
55733,2024-12-31 23:20:00+00:00,239.97,239.97,239.97,239.97,179.0
55734,2024-12-31 23:35:00+00:00,239.86,239.86,239.86,239.86,132.0


In [ ]:
ticker = "JPM"
data = yf.download(tickers = ticker, start="2023-06-01", end="2025-01-01", interval="1h")

Group by daily data, calculate difference and then predict. Use data of one week and neural netwrok for probability

In [10]:
data["Daily return"] = np.where(data["Close"] - data["Open"]  >= 0, 1, 0)

In [ ]:
data["Future daily return"] = data["Daily return"].shift(-1)

In [ ]:
data

In [ ]:
import numpy as np
import pandas as pd
import iisignature
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import norm
from tqdm.auto import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [3]:
def leadlag(X, use_time = False, print_arrays = False):
   if (not use_time):
       lead = []
       lag = []
       for val_lag, val_lead in zip(X[:-1], X[1:]):
           lead.append(val_lag)
           lag.append(val_lag)

           lead.append(val_lead)
           lag.append(val_lag)
       lead.append(X[-1])
       lag.append(X[-1])

       if (print_arrays):
           print(lead)
           print(lag)

       return np.c_[lead, lag]
   
   else:
       lead = []
       lag = []
       t = []
       time = 0
       t.append(time)
       lag.append(X[0])
       lead.append(X[0])

       for val_lag, val_lead in zip(X[:-1], X[1:]):

           time += 1

           t.append(time)
           lead.append(val_lag)
           lag.append(val_lag)

           t.append(time)
           lead.append(val_lead)
           lag.append(val_lag)

           t.append(time)
           lead.append(val_lead)
           lag.append(val_lead)

       if (print_arrays):
           print(lead)
           print(lag)
           print(t)

       return np.c_[lead, lag, t]

In [4]:
def generatePoisson(n):
   lam = 0.25
   events = np.random.poisson(lam, n) 
   counts = np.cumsum(events)
   return counts

def generateBrownian(n, delta=0.1):
   x = 0
   u = []
   for k in range(n):
       x = x + norm.rvs(scale=delta**2)
       u.append([x])
   return np.array(u)

In [5]:
def plot3D(path, recovered_path=None):
   fig = plt.figure()
   ax = fig.add_subplot(111, projection='3d')  # Create a 3D subplot

   # Plot the path
   ax.plot(path[:, 0], path[:, 1], path[:, 2], label='Original Path')

   # Optionally, plot the recovered path
   if recovered_path is not None:
       ax.plot(recovered_path[:, 0], recovered_path[:, 1], recovered_path[:, 2], label='Recovered Path')

   ax.set_xlabel('X')
   ax.set_ylabel('Y')
   ax.set_zlabel('Z')
   ax.set_title('3D Path')
   ax.legend()

   plt.show()

def add_time(X):
   return np.c_[X, list(range(len(X)))]

In [ ]:
n_points = 20
n_dims = 2
x = generateBrownian(n_points)
y = generatePoisson(n_points)
path = np.c_[x, y]
plot3D(add_time(path))

In [ ]:
def stream_to_logsig(X, order):
   X = np.array(X)
   s = iisignature.prepare(X.shape[1], order)
   return iisignature.logsig(X, s)

def data_transform(data):
   leadlag_path = leadlag(data)
   return leadlag_path

transformed_path = data_transform(path)
logsig = stream_to_logsig(transformed_path, 5)
print(len(logsig))
logsig = stream_to_logsig(transformed_path, 7)
print(len(logsig))
logsig = stream_to_logsig(transformed_path, 10)
print(len(logsig))

In [ ]:
clf = RandomForestClassifier()

path = pd.DataFrame()
pathSign = pd.DataFrame()

# Simulations
order = 10
for i in range(0, 500):
   transformed_path = data_transform(generateBrownian(70, 0.1))
   logsig = stream_to_logsig(transformed_path, order)
   pathSign = pathSign.append(pd.DataFrame(logsig.flatten(),  columns = [i]).T)

   path = path.append(pd.DataFrame(generateBrownian(70, 0.1).flatten(),  columns = [i]).T)

for i in range(501, 1000):
   transformed_path = data_transform(generateBrownian(70, 0.12))
   logsig = stream_to_logsig(transformed_path, order)
   pathSign = pathSign.append(pd.DataFrame(logsig.flatten(),  columns = [i]).T)

   path = path.append(pd.DataFrame(generateBrownian(70, 0.12).flatten(),  columns = [i]).T)

# Fitting
path['target'] = 0  # or pathSign
path.loc[501:, 'target'] = 1

X = path.drop('target', axis=1)
y = path['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Results
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))